NOTE: This is a Jupyter Notebook, so it won't be used in production, only for testing -> Be careful, as some changes in this file won't affect the production code and vice versa

Resources:
- https://quantum-computing.ibm.com/composer/files/new
- https://wybiral.github.io/quantum/
- https://jarrodmcclean.com/basic-quantum-circuit-simulation-in-python/
- https://quantumcomputing.stackexchange.com/questions/29454/why-do-quantum-computing-simulators-have-the-measurement-function

# Quantum Computation

In [85]:
import numpy as np

In [86]:
### basic stuff

# zero ket
ZKet = np.array([1,0])
# one ket
OKet = np.array([0,1])

### operator matrices

# identity
Id = np.array([
    [1,0],
    [0,1]
])
# pauli x
X = np.array([
    [0,1],
    [1,0]
])
# pauli y
Y = np.array([
    [0,0-1j],
    [0+1j,0]
])
# pauli z
Z = np.array([
    [1,0],
    [0,-1]
])
# hadamard
H = 1/np.sqrt(2) * np.array([
    [1,1],
    [1,-1]
])
# phase
S = np.array([
    [1,0],
    [0,0+1j]
])
# pi over eight
T = np.array([
    [1,0],
    [0,np.exp((0+1j)*np.pi/4)]
])
# |0><0>
P_0 = np.array([
    [1,0],
    [0,0]
])
# |1><1>
P_1 = np.array([
    [0,0],
    [0,1]
])

In [87]:
class QState:
    def __init__(self, qbit_cnt, info=False):
        self.qbit_cnt = qbit_cnt
        # create state |00...0> with qbit_cnt amount of zeros
        self.state = np.zeros(2**self.qbit_cnt, dtype=complex)
        self.state[0] = 1

        if info:
            max_bin_width = len(bin(len(self.state) - 1)[2:])
            binary_str = bin(0)[2:].zfill(max_bin_width)
            print("Initial state: |{}>".format(binary_str))
    
    # show all states including the impossible one (unless
    # otherwise noted) and show according probabilities
    # DISCLAIMER: rounding/floating point errors may occur
    def show_state_and_probs(self, reduced=False):
        max_len = 0
        for element in self.state:
            if reduced and element == 0+0j:
                continue

            curr_len = max(len(str(element.real)), len(str(element.imag)))
            if curr_len > max_len:
                max_len = curr_len
        
        max_bin_width = len(bin(len(self.state) - 1)[2:])

        for i, element in enumerate(self.state):
            if reduced and element == 0+0j:
                continue

            real_part_str = "{:.{width}f}".format(element.real, width=max_len)
            imaginary_part_str = "{:.{width}f}".format(element.imag, width=max_len)
            
            formatted_element = ("{}+{}i".format(real_part_str, imaginary_part_str) if element.imag >= 0 else
                                 "{}{}i".format(real_part_str, imaginary_part_str))

            binary_str = bin(i)[2:].zfill(max_bin_width)

            prob_str = str(round((np.absolute(element)**2)*100, 4))

            if i > 0 and element.real >= 0:
                print("+{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i > 0 and element.real < 0:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i <= 0 and element.real >= 0:    
                print(" {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            else:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))

    ###
    ### GATE CREATION HELPER ###
    ###
    
    # e.g. turns [A, B, C, D] into A ⊗ B ⊗ C ⊗ D
    def op_mats_arr_to_tens(self, op_mats_arr):
        result = np.array([[1.0+0j]], dtype=complex)
        for op_mat in op_mats_arr:
            result = np.kron(result, op_mat)
        
        return result

    def sing_qbit_op(self, pos, op_mat):
        op_mats_arr = (
              (pos)*[Id]
            + (1)*[op_mat]
            + (self.qbit_cnt-pos-1)*[Id]
        )
        gate = self.op_mats_arr_to_tens(op_mats_arr)
        self.state = np.matmul(gate, self.state)
    
    # e.g. turns np.array([a, b, c, d])
    # into [[a, ZKet, ZKet], [b, ZKet, OKet], [c, OKet, ZKet], [d, OKet, OKet]]
    # both represent a*ZKET⊗ZKet + b*ZKet⊗OKet + c*OKet⊗ZKet + d*OKet⊗OKet
    def decomp_state(self, comp):
        decomp = []

        max_bin_width = len(bin(len(comp) - 1)[2:])
        for i in range(len(comp)):
            binary_str = bin(i)[2:].zfill(max_bin_width)
            
            sub_decomp = [comp[i]]
            for letter in binary_str:
                if letter == "0":
                    sub_decomp.append(ZKet)
                elif letter == "1":
                    sub_decomp.append(OKet)
            decomp.append(sub_decomp)
        
        return decomp
    
    # e.g. turns [[a, ZKet, ZKet], [b, ZKet, OKet], [c, OKet, ZKet], [d, OKet, OKet]]
    # into np.array([a, b, c, d])
    # both represent a*ZKET⊗ZKet + b*ZKet⊗OKet + c*OKet⊗ZKet + d*OKet⊗OKet
    def comp_state(self, decomp):
        state = np.zeros(2**(len(decomp[0])-1), dtype=complex)

        for element in decomp:
            state += element[0]*self.op_mats_arr_to_tens(element[1:])[0]

        return state

    # decompose state into computation basis form, e.g.: [a,b,c,d] -> a|00>+b|01>+c|10>+d|11>
    # then go over each summand and check if the qubit at ctrl_pos is set to one
    # if no, just continue
    # if yes, apply the target function to the summand -> add result to a temporary variable
    # when looped over all summands, convert the temporary variable back to a normal state [w,x,y,z]
    # which then becomes the main state
    def ctrl(self, ctrl_pos, targ_func, *targ_pos_args):
        decomp = self.decomp_state(self.state)
        final_decomp = decomp.copy()

        for i in range(len(decomp)):
            for j in range(1, len(decomp[i])):
                if j-1 != ctrl_pos or np.array_equal(decomp[i][j], ZKet):
                    continue
                
                orig_qbit_cnt = self.qbit_cnt
                self.qbit_cnt = len(decomp[i])-1

                orig_state = self.state
                self.state = self.comp_state([decomp[i]])
                targ_func(*targ_pos_args)
                # the following one line is the workaround since `final_decomp.remove(decomp[i])` doesn't work
                final_decomp = [item for item in final_decomp if not (all(np.array_equal(xi, yi) for xi, yi in zip(item, decomp[i])))]
                final_decomp += self.decomp_state(self.state)
                
                self.qbit_cnt = orig_qbit_cnt
                self.state = orig_state
                break # out of second for loop

        self.state = self.comp_state(final_decomp)

    def ctrl_ctrl(self, ctrl_pos1, ctrl_pos2, targ_func, *targ_pos_args):
        decomp = self.decomp_state(self.state)
        final_decomp = decomp.copy()

        for i in range(len(decomp)):
            ctrl_pos1_set = False
            ctrl_pos2_set = False
            for j in range(1, len(decomp[i])):
                if j-1 == ctrl_pos1 and np.array_equal(decomp[i][j], OKet):
                    ctrl_pos1_set = True
                elif j-1 == ctrl_pos2 and np.array_equal(decomp[i][j], OKet):
                    ctrl_pos2_set = True
                
                if (not ctrl_pos1_set) or (not ctrl_pos2_set):
                    continue
                
                orig_qbit_cnt = self.qbit_cnt
                self.qbit_cnt = len(decomp[i])-1

                orig_state = self.state
                self.state = self.comp_state([decomp[i]])
                targ_func(*targ_pos_args)
                # the following one line is the workaround since `final_decomp.remove(decomp[i])` doesn't work
                final_decomp = [item for item in final_decomp if not (all(np.array_equal(xi, yi) for xi, yi in zip(item, decomp[i])))]
                final_decomp += self.decomp_state(self.state)
                
                self.qbit_cnt = orig_qbit_cnt
                self.state = orig_state
                break # out of second for loop

        self.state = self.comp_state(final_decomp)
     
    ###
    ### SINGLE-QUBIT GATES ###
    ###

    def pauli_x(self, pos):
        self.sing_qbit_op(pos, X)

    def pauli_y(self, pos):
        self.sing_qbit_op(pos, Y)

    def pauli_z(self, pos):
        self.sing_qbit_op(pos, Z)

    def hadamard(self, pos):
        self.sing_qbit_op(pos, H)
    
    def phase(self, pos):
        self.sing_qbit_op(pos, S)

    def pi_ov_8(self, pos):
        self.sing_qbit_op(pos, T)

    ###
    ### MULTI-QUBIT GATES ###
    ###

    # doesn't matter whether ctrl_pos smaller or bigger than targ_pos
    def cnot(self, ctrl_pos, targ_pos):
        self.ctrl(ctrl_pos, self.pauli_x, targ_pos)

    def cz(self, ctrl_pos, targ_pos):
        self.ctrl(ctrl_pos, self.pauli_z, targ_pos)
    
    def cp(self, ctrl_pos, targ_pos):
        self.ctrl(ctrl_pos, self.phase, targ_pos)
    
    def toffoli(self, ctrl_pos1, ctrl_pos2, targ_pos):
        self.ctrl_ctrl(ctrl_pos1, ctrl_pos2, self.pauli_x, targ_pos)

    def swap(self, pos1, pos2):
        # using special decomposition to achieve it's functionality:
        self.cnot(pos1, pos2)
        self.cnot(pos2, pos1)
        self.cnot(pos1, pos2)

    def fredkin(self, ctrl_pos, swap_pos1, swap_pos2):
        self.ctrl(ctrl_pos, self.swap, swap_pos1, swap_pos2)



# Testing

In [88]:
### Testing

qs = QState(4, True)

qs.pauli_x(1)
qs.pauli_x(2)
qs.hadamard(0)
qs.hadamard(3)
qs.cnot(0,2)

qs.show_state_and_probs(True)

# EXPECTED:
# 0.50000000+0.00000000i 	|0110> 	25.0000%
# 0.50000000+0.00000000i 	|0111> 	25.0000%
# 0.50000000+0.00000000i 	|1100> 	25.0000%
# 0.50000000+0.00000000i 	|1101> 	25.0000%
# SUCCESS!

Initial state: |0000>
+0.499999999999999889+0.000000000000000000i|0110>	-> 25.0%
+0.499999999999999889+0.000000000000000000i|0111>	-> 25.0%
+0.499999999999999889+0.000000000000000000i|1100>	-> 25.0%
+0.499999999999999889+0.000000000000000000i|1101>	-> 25.0%


In [89]:
qs = QState(2, True)

qs.hadamard(0)
qs.pauli_x(1)
qs.cz(0,1)

qs.show_state_and_probs(True)

# EXPECTED
# 0.70710678+0.00000000i 	|01> 	50.0000%
#-0.70710678+0.00000000i 	|11> 	50.0000%
# SUCCESS!

Initial state: |00>
+0.7071067811865474617+0.0000000000000000000i|01>	-> 50.0%
-0.7071067811865474617+0.0000000000000000000i|11>	-> 50.0%


In [90]:
qs = QState(2, True)

qs.hadamard(0)
qs.cnot(0,1)

qs.show_state_and_probs(True)

# EXPECTED
# 0.70710678+0.00000000i 	|00> 	50.0000%
# 0.70710678+0.00000000i 	|11> 	50.0000%
# SUCCESS!

Initial state: |00>
 0.707106781186547462+0.000000000000000000i|00>	-> 50.0%
+0.707106781186547462+0.000000000000000000i|11>	-> 50.0%


In [91]:
qs = QState(2, True)

qs.pauli_x(0)
qs.swap(0,1)

qs.show_state_and_probs(True)

# EXPECTED
# 1.00000000+0.00000000i 	|01> 	100.0000%
# SUCCESS!

Initial state: |00>
+1.000+0.000i|01>	-> 100.0%


In [92]:
qs = QState(3, True)

qs.pauli_x(0)
qs.pauli_x(1)

qs.fredkin(0,1,2)

qs.show_state_and_probs(True)

# EXPECTED
# 1.00000000+0.00000000i 	|101> 	100.0000%
# SUCCESS!

Initial state: |000>
+1.000+0.000i|101>	-> 100.0%


In [93]:
qs = QState(3, True)

qs.pauli_x(0)
qs.pauli_x(1)

qs.fredkin(1,0,2)

qs.show_state_and_probs(True)

# EXPECTED
# 1.00000000+0.00000000i 	|011> 	100.0000%
# SUCCESS!

Initial state: |000>
+1.000+0.000i|011>	-> 100.0%


In [94]:
qs = QState(3, True)

qs.pauli_x(0)
qs.pauli_x(1)

qs.toffoli(0,1,2)

qs.show_state_and_probs(True)

# EXPECTED
# 1.00000000+0.00000000i 	|111> 	100.0000%
# SUCCESS!

Initial state: |000>
+1.000+0.000i|111>	-> 100.0%


In [95]:
qs = QState(2, True)

qs.pauli_x(0)
qs.hadamard(1)
qs.pauli_z(0)

qs.show_state_and_probs(True)

# EXPECTED
# -0.70710678+0.00000000i 	|10> 	50.0000%
# -0.70710678+0.00000000i 	|11> 	50.0000%
# SUCCESS!

Initial state: |00>
-0.7071067811865474617+0.0000000000000000000i|10>	-> 50.0%
-0.7071067811865474617+0.0000000000000000000i|11>	-> 50.0%


In [96]:
qs = QState(5, True)

qs.pauli_x(0)
qs.pauli_x(1)
qs.toffoli(0,1,2)
qs.hadamard(3)
qs.hadamard(4)

qs.show_state_and_probs(True)

# EXPECTED
# 0.00000000+0.50000000i 	|11000> 	25.0000%
# 0.00000000+0.50000000i 	|11001> 	25.0000%
# 0.00000000+0.50000000i 	|11010> 	25.0000%
# 0.00000000+0.50000000i 	|11011> 	25.0000%
# SUCCESS!

Initial state: |00000>
+0.499999999999999889+0.000000000000000000i|11100>	-> 25.0%
+0.499999999999999889+0.000000000000000000i|11101>	-> 25.0%
+0.499999999999999889+0.000000000000000000i|11110>	-> 25.0%
+0.499999999999999889+0.000000000000000000i|11111>	-> 25.0%


In [98]:
foos = [0, 545, 1, 3]
new_foos = 

for foo in range(len(foos)):
    print(foos[i])
    if foos[i] == 1:
        foos.pop(i)
        i += 1

print(foos)

0
545
1


IndexError: list index out of range

# Protocol

In [ ]:
input_info = {"qbit_cnt": 3}
input_gates = [{"type": "pauli_x", "qbit": [0]}, {"type": "cnot", "qbit": [0, 1]}]
# careful that in [0, 1] the first number MUST be the control qubit, and the other one the target qubit
# SO EVERYTHING MUST BE STRUCTURED ACCORDING TO THE GATE FUNCTIONS (that are listed in QState) 

qs = QState(input_info["qbit_cnt"], True)

for input_gate in input_gates:
    gate_func = getattr(qs, input_gate["type"])
    gate_func(*input_gate["qbit"])

qs.show_state_and_probs(True)

Initial state: |000>
+1.000+0.000i|110>	-> 100.0%
